In [ ]:
import os
import sys
import time as t_util
import numpy as np
import cftime
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import yaml


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:    path_main  = file.read()
with open('../path_gwls.txt', 'r') as file:    path_gwls  = file.read()
    
dir_CMIP5    = f'{path_main}Data/CMIP5/HSIs/'
dir_GWL      = f'{path_gwls}cmip5_all_ens/'
dir_EMT      = f'{path_main}Data/CMIP5/EMT/'
dir_names    = f'{path_main}Scripts/Model_lists/'
dir_out      = f'{path_main}Data/Plot_preparation/Threshold_Exceedance/CMIP5/'
if not os.path.exists(dir_out): os.mkdir(dir_out)


## Prepare variables and parameters

In [ ]:
#Define cities
cities = ['Lisbon', 'Madrid', 'Barcelona', 'Rome', 'Athens', 'Istanbul', 'Sofia', 'Bucharest', 'Belgrade', 'Zagreb',
          'Milan', 'Budapest', 'Munich', 'Vienna', 'Prague', 'Paris', 'Brussels', 'Amsterdam', 'London', 'Dublin',
          'Hamburg', 'Copenhagen', 'Berlin', 'Warsaw', 'Kharkiv', 'Kyiv', 'Minsk', 'Vilnius', 'Riga', 'Moscow',
          'NizhnyNovgorod', 'Kazan', 'SaintPetersburg', 'Helsinki', 'Stockholm', 'Oslo']

#Define HSIs
HSI_names = ['TX']

#Define models and RCPs which should be used
all_models = dict()
all_models['rcp85'] = []
with open(dir_names + 'Models_CMIP5_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp85'].append(line[:-1])

        #Get model names (including member)
dir_data = dir_CMIP5 + 'Berlin/rcp85/'
model_names = sorted(list(set(["_".join(file.split('_')[2:4]) for file in os.listdir(dir_data)])))
        
#Read warming levels
fname = dir_GWL + 'cmip5_warming_levels_all_ens_1850_1900_no_bounds_check.yml'
with open(fname, 'r') as file:
    GWL_data = yaml.safe_load(file)
    
#Define warming levels
GWL_levels = ['10', '20', '30']


## Define thresholds

In [ ]:
#Level 1: warm (caution)
#Level 2: hot (extreme caution)
#Level 3: very_hot (danger)
#Level 4: sweltering (extreme danger)
thresholds = {}

#Tropical nights (and sensitivity tests)
thresholds["TN_Level1"] = 15
thresholds["TN_Level2"] = 17
thresholds["TN_Level3"] = 20
thresholds["TN_Level4"] = 23

#TX
thresholds["TX_Level1"] = 25
thresholds["TX_Level2"] = 27
thresholds["TX_Level3"] = 30
thresholds["TX_Level4"] = 33

#Names for NetCDF file
threshold_names = ["Level1", "Level2", "Level3", "Level4"]


## Prepare data according to European warming (EMT)

In [ ]:
#Define RCP
RCP = 'rcp85'
scenarios = ['historical', RCP]

EMT_change     = np.array([1.0, 2.0, 3.0])
EMT_change_str = ['1.0K', '2.0K', '3.0K']

THR_levels = ['Level1', 'Level2', 'Level3', 'Level4']

#Create empty array for storing heat stress indicators
da_empty = np.zeros((len(model_names), len(cities), len(EMT_change), len(THR_levels))) * np.NaN
data_coll = xr.Dataset(coords={'model':      ('model', model_names),
                               'city':       ('city', cities),
                               'EMT_change': ('EMT_change', EMT_change_str),
                               'THR_level':  ('THR_level', THR_levels)})

for HSI in HSI_names:  data_coll[HSI] = (('model', 'city', 'EMT_change', 'THR_level'), da_empty.copy())

#Loop over cities
for city in cities:
    
    print(city, end=', ')

    #Loop over models
    for i1, model in enumerate(all_models[RCP]):

        #Define members
        if model=='EC-EARTH':      members = ['r1i1p1', 'r12i1p1']
        elif model=='MPI-ESM-LR':  members = ['r1i1p1', 'r2i1p1', 'r3i1p1']
        else:                      members = ['r1i1p1']        
        
        #Loop over members
        for member in members:

            #Define name for storing data in array
            model_name = model + '_' + member
            

            #Loop over scenarios
            create = 1
            for scen in scenarios:

                #Define folder
                dir_data = dir_CMIP5 + city + '/' + scen + '/'

                #Read data
                fname_CMIP5 = [file for file in os.listdir(dir_data) if model + '_' in file and member in file]
                if len(fname_CMIP5)!=1:  sys.exit('File is not unique')
                data_read = xr.open_dataset(dir_data + fname_CMIP5[0])
                data_read = data_read

                #Concatenate data
                if create==1:
                    data_CMIP5 = data_read
                    create = 0
                else:
                    data_CMIP5 = xr.concat((data_CMIP5, data_read), dim='time')

            #Read European mean temperature (EMT)
            files_EMT = sorted([dir_EMT + file for file in os.listdir(dir_EMT) if model + '_' in file and 'EMT_' in file and member in file])
            data_EMT  = xr.concat((xr.open_dataset(file) for file in files_EMT), dim='time')

            #Calculate EMT relative to 1981-2010 and calculate 20-year means
            dataEMT_ref = data_EMT.sel(time=slice('1981', '2010')).mean('time')
            dataEMT_rel = data_EMT - dataEMT_ref
            dataEMT_20y = dataEMT_rel.rolling(time=20, center=True).mean()

            #Loop over selected EMT levels
            for dEMT, dEMT_str in zip(EMT_change, EMT_change_str):

                #Identify 20-year period in wich level is reached for first time
                dEMT_ind  = np.where(dataEMT_20y.tas>dEMT)
                if len(dEMT_ind[0])==0:
                    if city==cities[0]:  print("  " + dEMT_str + ' not reached in ' + model)
                    continue
                    
                ind = dEMT_ind[0][0]
                central_year = dataEMT_20y.isel(time=ind).time.dt.year
                start_year   = int(central_year - 20 / 2)
                end_year     = int(central_year + (20 / 2 - 1))
                years_sel    = slice(str(start_year), str(end_year))

                #Get data in 20y-period
                data_20y = data_CMIP5.sel(time=years_sel)

                #Loop over HSIs
                for HSI in HSI_names:

                    #Correct TN and TX
                    if HSI in ['TN', 'TX']:  data_sel = data_20y[HSI] - 273.15
                    else:                    data_sel = data_20y[HSI]

                    #Loop over levels
                    for THR_level in THR_levels:

                        #Calculate exceedance
                        data_exc = data_sel>=thresholds[HSI + "_" + THR_level]
                        data_exc = data_exc.sum() / (end_year - start_year + 1)

                        #Put data in array
                        for HSI in HSI_names:
                            data_coll[HSI].loc[{"model": model_name, "city": city, "EMT_change": dEMT_str, "THR_level": THR_level}] = data_exc
                    
#Save data in file
fname_out = dir_out + 'HSIs-ThresholdExceedance_' + RCP + '_EMT.nc'
if os.path.exists(fname_out): os.remove(fname_out)
data_coll.to_netcdf(fname_out)


## Prepare GWL data

In [ ]:
#Define RCP
RCP = 'rcp85'
scenarios = ['historical', RCP]

THR_levels = ['Level1', 'Level2', 'Level3', 'Level4']

#Create empty array for storing heat stress indicators
da_empty = np.zeros((len(model_names), len(cities), len(GWL_levels), len(THR_levels))) * np.NaN
data_coll = xr.Dataset(coords={'model':      ('model', model_names),
                               'city':       ('city', cities),
                               'GWL_level':  ('GWL_level', GWL_levels),
                               'THR_level':  ('THR_level', THR_levels)})

for HSI in HSI_names:  data_coll[HSI] = (('model', 'city', 'GWL_level', 'THR_level'), da_empty.copy())

#Loop over cities
for city in cities:
    
    print(city, end=', ')

    #Loop over models
    for i1, model in enumerate(all_models[RCP]):

        #Define members
        if model=='EC-EARTH':      members = ['r1i1p1', 'r12i1p1']
        elif model=='MPI-ESM-LR':  members = ['r1i1p1', 'r2i1p1', 'r3i1p1']
        else:                      members = ['r1i1p1']        
        
        #Loop over members
        for member in members:

            #Define name for storing data in array
            model_name = model + '_' + member

            #Loop over scenarios
            create = 1
            for scen in scenarios:

                #Define folder
                dir_data = dir_CMIP5 + city + '/' + scen + '/'

                #Read data
                fname_CMIP5 = [file for file in os.listdir(dir_data) if model + '_' in file and member in file]
                if len(fname_CMIP5)!=1:  sys.exit('File is not unique')
                data_read = xr.open_dataset(dir_data + fname_CMIP5[0])
                data_read = data_read

                #Concatenate data
                if create==1:
                    data_CMIP5 = data_read
                    create = 0
                else:
                    data_CMIP5 = xr.concat((data_CMIP5, data_read), dim='time')

            #Read time periods when certain global warming levels (GWL) are reached
            time_GWL = dict()
            for GWL_level in GWL_levels:
                data_level = GWL_data['warming_level_' + GWL_level]
                entry_sel = [entry for entry in data_level if entry['model']==model and entry['exp']==RCP and entry['ensemble']==member]

                #Select data in time period when GWL is reached
                if len(entry_sel)==1:
                    time_GWL = slice(str(entry_sel[0]['start_year']), str(entry_sel[0]['end_year']))
                else:
                    if city==cities[0]:  print(str(int(GWL_level)/10) + 'K not reached in ' + model)
                    continue              

                #Select data in time period when GWL is reached
                start_year = entry_sel[0]['start_year']
                end_year   = entry_sel[0]['end_year']

                #Get data in 20y-period
                data_20y = data_CMIP5.sel(time=time_GWL)

                #Loop over HSIs
                for HSI in HSI_names:

                    #Correct TN and TX
                    if HSI in ['TN', 'TX']:  data_sel = data_20y[HSI] - 273.15
                    else:                    data_sel = data_20y[HSI]

                    #Loop over levels
                    for THR_level in THR_levels:

                        #Calculate exceedance
                        data_exc = data_sel>=thresholds[HSI + "_" + THR_level]
                        data_exc = data_exc.sum() / (end_year - start_year + 1)

                        #Put data in array
                        for HSI in HSI_names:
                            data_coll[HSI].loc[{"model": model_name, "city": city, "GWL_level": GWL_level, "THR_level": THR_level}] = data_exc
                    
#Save data in file
fname_out = dir_out + 'HSIs-ThresholdExceedance_' + RCP + '_GWL.nc'
if os.path.exists(fname_out): os.remove(fname_out)
data_coll.to_netcdf(fname_out)


## Prepare time period data

In [ ]:
#Select time periods
time_periods = [[1981, 2010],
                [2036, 2065],
                [2070, 2099]]

#Define time string
time_strings = [str(time[0]) + '-' + str(time[1]) for time in time_periods]

THR_levels = ['Level1', 'Level2', 'Level3', 'Level4']

#Define RCP
RCP = 'rcp85'
scenarios = ['historical', RCP]

#Create empty array for storing heat stress indicators
da_empty = np.zeros((len(model_names), len(cities), len(time_periods), len(THR_levels))) * np.NaN
data_coll = xr.Dataset(coords={'model':       ('model', model_names),
                               'city':        ('city', cities),
                               'time_period': ('time_period', time_strings),
                               'THR_level':   ('THR_level', THR_levels)})

for HSI in HSI_names:  data_coll[HSI] = (('model', 'city', 'time_period', 'THR_level'), da_empty.copy())

#Loop over cities
for city in cities:
    
    print(city, end=', ')

    #Loop over models
    for i1, model in enumerate(all_models[RCP]):

        #Define members
        if model=='EC-EARTH':      members = ['r1i1p1', 'r12i1p1']
        elif model=='MPI-ESM-LR':  members = ['r1i1p1', 'r2i1p1', 'r3i1p1']
        else:                      members = ['r1i1p1']        
        
        #Loop over members
        for member in members:

            #Define name for storing data in array
            model_name = model + '_' + member
            
            #Loop over scenarios
            create = 1
            for scen in scenarios:

                #Define folder
                dir_data = dir_CMIP5 + city + '/' + scen + '/'

                #Read data
                fname_CMIP5 = [file for file in os.listdir(dir_data) if model + '_' in file and member in file]
                if len(fname_CMIP5)!=1:  sys.exit('File is not unique')
                data_read = xr.open_dataset(dir_data + fname_CMIP5[0])
                data_read = data_read

                #Concatenate data
                if create==1:
                    data_CMIP5 = data_read
                    create = 0
                else:
                    data_CMIP5 = xr.concat((data_CMIP5, data_read), dim='time')

            #Loop over time periods
            for time_period, time_string in zip(time_periods, time_strings):

                #Select data in time period
                time_sel = slice(str(time_period[0]), str(time_period[1]))

                #Get data in 20y-period
                data_20y = data_CMIP5.sel(time=time_sel)

                #Loop over HSIs
                for HSI in HSI_names:

                    #Correct TN and TX
                    if HSI in ['TN', 'TX']:  data_sel = data_20y[HSI] - 273.15
                    else:                    data_sel = data_20y[HSI]

                    #Loop over levels
                    for THR_level in THR_levels:

                        #Calculate exceedance
                        data_exc = data_sel>=thresholds[HSI + "_" + THR_level]
                        data_exc = data_exc.sum() / (end_year - start_year + 1)

                        #Save in array
                        data_coll[HSI].loc[{"model": model_name, "city": city, "time_period": time_string, "THR_level": THR_level}] = data_exc

#Save data in file
fname_out = dir_out + 'HSIs-ThresholdExceedance_' + RCP + '_time-periods.nc'
if os.path.exists(fname_out): os.remove(fname_out)
data_coll.to_netcdf(fname_out)
